In [1]:
from os import listdir
from os.path import isfile, join
import subprocess
patient_folder = "../generated_patients_finetuned/"
onlyfiles = [f for f in listdir(patient_folder) if isfile(join(patient_folder, f))]

In [2]:
onlyfiles

['guideline_gc.jsonl_435.json',
 'guideline_mayo.jsonl_682.json',
 'guideline_mayo.jsonl_310.json',
 'guideline_gc.jsonl_709.json',
 'guideline_gc.jsonl_946.json',
 'guideline_nice.jsonl_858.json',
 'guideline_gc.jsonl_414.json',
 'guideline_rch.jsonl_265.json',
 'guideline_cdc_diseases.jsonl_15.json',
 'guideline_gc.jsonl_517.json',
 'guideline_mayo.jsonl_886.json',
 'guideline_gc.jsonl_576.json',
 'guideline_gc.jsonl_398.json',
 'guideline_nice.jsonl_1543.json',
 'guideline_mayo.jsonl_238.json',
 'guideline_mayo.jsonl_863.json',
 'guideline_gc.jsonl_599.json',
 'guideline_cdc_diseases.jsonl_3.json',
 'guideline_rch.jsonl_208.json',
 'guideline_gc.jsonl_710.json',
 'guideline_mayo.jsonl_211.json',
 'guideline_gc.jsonl_53.json',
 'guideline_rch.jsonl_153.json',
 'guideline_gc.jsonl_833.json',
 'guideline_gc.jsonl_826.json',
 'guideline_mayo.jsonl_288.json',
 'guideline_rch.jsonl_136.json',
 'guideline_mayo.jsonl_1096.json',
 'guideline_cdc_diseases.jsonl_29.json',
 'guideline_rch.jsonl

In [3]:
import json
content = []
for file in onlyfiles:
    with open(patient_folder+file) as f:
        a = json.load(f)
        for label, text in a.items():
            label = label[:-2]
            content += [(text, label, file)]
            print(file)

guideline_gc.jsonl_435.json
guideline_gc.jsonl_435.json
guideline_gc.jsonl_435.json
guideline_mayo.jsonl_682.json
guideline_mayo.jsonl_682.json
guideline_mayo.jsonl_682.json
guideline_mayo.jsonl_310.json
guideline_mayo.jsonl_310.json
guideline_mayo.jsonl_310.json
guideline_gc.jsonl_709.json
guideline_gc.jsonl_709.json
guideline_gc.jsonl_709.json
guideline_gc.jsonl_946.json
guideline_gc.jsonl_946.json
guideline_gc.jsonl_946.json
guideline_nice.jsonl_858.json
guideline_nice.jsonl_858.json
guideline_nice.jsonl_858.json
guideline_gc.jsonl_414.json
guideline_gc.jsonl_414.json
guideline_gc.jsonl_414.json
guideline_rch.jsonl_265.json
guideline_rch.jsonl_265.json
guideline_rch.jsonl_265.json
guideline_cdc_diseases.jsonl_15.json
guideline_cdc_diseases.jsonl_15.json
guideline_cdc_diseases.jsonl_15.json
guideline_gc.jsonl_517.json
guideline_gc.jsonl_517.json
guideline_gc.jsonl_517.json
guideline_mayo.jsonl_886.json
guideline_mayo.jsonl_886.json
guideline_mayo.jsonl_886.json
guideline_gc.jsonl_576

In [4]:
content[0]

("Patient Description 1:\nA 6-year-old patient, with no significant medical history, was brought in by their parents for an ophthalmologic evaluation. The child has been experiencing a persistent reduced visual acuity in one eye, which the parents noticed during daily activities such as reading or watching television. Despite interventions, including adequate corrective lenses, the visual acuity issue persists. The child also exhibits poor depth perception, often struggling with tasks that require hand-eye coordination. Furthermore, the parents have observed the child squinting or closing one eye to see better, particularly when engaged in activities that demand focused visual attention. The diagnostic process may be influenced by the child's age and level of cooperation during the assessment.",
 'Amblyopia',
 'guideline_gc.jsonl_435.json')

In [5]:
def remove_newlines(text):
    return text.replace('\n', ' ')

def save_to_file(text, filename):
    with open(filename, 'w') as file:
        file.write(text)

In [6]:
def run_mistral_with_input(input_text, timeout):
    process = subprocess.Popen(['ollama', 'run', 'mistral'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    try:
        stdout, stderr = process.communicate(input_text, timeout=timeout)
        
        if process.returncode != 0:
            print(f"Error executing command: {stderr}")
            return None

    except subprocess.TimeoutExpired:
        process.terminate()
        process.wait()
        print("Process timed out and was terminated.")
        return None

    finally:
        if process.poll() is None: 
            process.terminate()
        process.wait()
    return stdout

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "mistralai/Mistral-7B-v0.1"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

KeyboardInterrupt: 

In [35]:
for (text,label, file) in content:
    input_text = " Here is a patient description : " +text+ """
Fill the following structure accordingly. Don't change it.
If the information is not given, don't write anything, leave it as an empty section.
If an information is true for multiple symptoms, repeat the information.
Only answer with the filled structure.

{
  "symptoms": [
    {
      "name of the symptom": "",
      "intensity of symptom": "",
      "specific attributes of the symptom": {
        "location": "",
        "size": "",
        "color": "",
        "frequency": ""
      },
      "When did the symptom appear ": "",
      "previous treatments": "",
      "reaction to previous treaments"
      "behaviour affecting the symptom": ""
    }
  ],
  "socio economic context": {
  },
  "geographic_context": {
      "recent travels": "",
      "level of care": ""
  },
  "physiological context": {
  
  },
  "psychological context": {
  
  },
  "personal medical_history": {
  
  },
  "family medical_history": {
  
  },
  "current medication": {

  },
  "lifestyle factors": {
  
  },
  "results of recent medical tests": {
  
  }
}
<END>
"""

    #input_text = remove_newlines(input_text)
    
    device = "cpu"
    
    model_inputs = tokenizer([input_text], return_tensors="pt").to(device)

    model.to(device)

    generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)

    response = tokenizer.batch_decode(generated_ids)[0]
    
    #response = run_mistral_with_input(input_text, 60 * 5)
    print(response)
    if response:
        with open("train/structured_patient/" + file[:-4], "w") as file:
            file.write(response)
        print("Mistral's response saved to response.txt")

>>>  Here is a patient description :     Patient A, a 45-year-old male, presents with a history of chronic pain in the left lower extremity, which he attributes to a previous injury sustained during a sporting activity. The pain is described as persistent, scoring an average of 7/10 on the visual analog scale, and has been minimally responsive to oral analgesics and physical therapy. Additionally, he experiences episodes of sharp shooting pain radiating down his left leg. Physical examination reveals sensory deficits and reduced proprioception in the affected limb. Given the patient's chronic pain and the inadequacy of conventional therapies, there is an indication for the evaluation of regional nerve block as a management strategy. The diagnostic and treatment decisions will be influenced by the clinician's experience in administering regional nerve blocks, training in ultrasound guidance techniques, and the quality of available ultrasound equipment. Fill the following structure accor

KeyboardInterrupt: 

In [14]:
run_mistral_with_input("Salut ?", timeout = 60)

'>>> Salut ?\nSalut! Comment allez-vous ?\n\n'